In [ ]:
import os
import pathlib
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D

In [ ]:
base_path = '/content/drive/MyDrive/plant_village_dataset/plant_village_dataset/'


In [ ]:
# import os

# # Path to the 'color' folder in Google Drive
# color_folder_path = base_path + 'color'

# # Dictionary to store the count of images in each subfolder
# image_counts = {}

# # Walk through each subfolder in the 'color' folder
# for folder_name in os.listdir(color_folder_path):
#     folder_path = os.path.join(color_folder_path, folder_name)
#     if os.path.isdir(folder_path):  # Ensure it's a folder
#         # Count the number of files in the folder
#         image_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
#         image_counts[folder_name] = image_count

# # Display the results
# for folder, count in image_counts.items():
#     print(f"{folder}: {count} images")

# # Optionally, show the total number of images across all folders
# total_images = sum(image_counts.values())
# print(f"\nTotal images across all folders: {total_images}")


In [ ]:
# import os
# import shutil
# import random

# # Source and target directories
# source_folder = base_path+ 'color'
# target_base = base_path+ 'split'

# # Subdirectories for train, test, and val
# train_folder = os.path.join(target_base, 'train')
# test_folder = os.path.join(target_base, 'test')
# val_folder = os.path.join(target_base, 'val')

# # Ensure target directories exist
# for folder in [train_folder, test_folder, val_folder]:
#     os.makedirs(folder, exist_ok=True)

In [ ]:
# # Split percentages
# train_split = 0.6
# test_split = 0.2
# val_split = 0.2

# # Iterate over each labeled folder
# for label_folder in os.listdir(source_folder):
#     label_path = os.path.join(source_folder, label_folder)
#     if os.path.isdir(label_path):
#         # Get all images in the current folder
#         images = os.listdir(label_path)
#         random.shuffle(images)

#         # Calculate split sizes
#         total_images = len(images)
#         train_size = int(total_images * train_split)
#         test_size = int(total_images * test_split)

#         # Split images
#         train_images = images[:train_size]
#         test_images = images[train_size:train_size + test_size]
#         val_images = images[train_size + test_size:]

#         # Define target subdirectories for the current label
#         train_label_folder = os.path.join(train_folder, label_folder)
#         test_label_folder = os.path.join(test_folder, label_folder)
#         val_label_folder = os.path.join(val_folder, label_folder)

#         # Create subdirectories
#         for folder in [train_label_folder, test_label_folder, val_label_folder]:
#             os.makedirs(folder, exist_ok=True)

#         # Move images to their respective folders
#         for img in train_images:
#             shutil.copy(os.path.join(label_path, img), os.path.join(train_label_folder, img))
#         for img in test_images:
#             shutil.copy(os.path.join(label_path, img), os.path.join(test_label_folder, img))
#         for img in val_images:
#             shutil.copy(os.path.join(label_path, img), os.path.join(val_label_folder, img))

#         print(f"Processed {label_folder}: Train={len(train_images)}, Test={len(test_images)}, Val={len(val_images)}")

# print("Dataset split and organization complete!")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import zipfile
# import os

# # Define the path to the zip file in Google Drive
# zip_path = '/content/drive/MyDrive/plant_village_dataset.zip'

# # Define the extraction path
# extract_path = '/content/drive/MyDrive/plant_village_dataset'

# # Unzip the dataset
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# # Verify the extraction
# print("Files extracted to:", os.listdir(extract_path))


In [ ]:
img_size, batch_size, shuffle_size = 224, 4, 1000
img_shape = (img_size, img_size, 3)

In [ ]:
train_dir = base_path + 'split/train'
val_dir = base_path + 'split/val'
test_dir = base_path + 'split/test'
model_save_path = base_path + 'model_checkpoints/1-27-2025-no-augmentation'

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 32571 images belonging to 38 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 10885 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.


In [ ]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.TPUStrategy(resolver)
# # Initialize the model within the strategy scope
# with strategy.scope():
#     base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=img_shape)
#     x = base_model.output
#     x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     predictions = tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')(x)
#     model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

#     # Freeze base layers
#     for layer in base_model.layers:
#         layer.trainable = False

#     # Compile the model
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=img_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# # Freeze base model layers
# for layer in base_model.layers:
#     layer.trainable = False


In [ ]:
# # Compile the model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [ ]:
# Callbacks
checkpoint = ModelCheckpoint(
    filepath=f"{model_save_path}/mobilenet_epoch{{epoch:02d}}.keras",
    save_best_only=False,
    save_freq=100,
    verbose=1
)

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    mode='max',
    restore_best_weights=True,
    verbose=1
)

In [ ]:
reduced_steps_per_epoch = 500
validation_steps = 200
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=2,  # Max epochs
    callbacks=[checkpoint, early_stopping],
    # steps_per_epoch=len(train_generator),
    # validation_steps=len(val_generator)
    steps_per_epoch=reduced_steps_per_epoch,
    validation_steps=validation_steps
)

# Evaluate on Test Data
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the final trained model
model.save(f"{model_save_path}/final_mobilenet_model.keras")
print("Model saved.")

Epoch 1/2
 99/500 ━━━━━━━━━━━━━━━━━━━━ 9:42 1s/step - accuracy: 0.2468 - loss: 3.0683
Epoch 1: saving model to /content/drive/MyDrive/plant_village_dataset/plant_village_dataset/model_checkpoints/1-27-2025-no-augmentation/mobilenet_epoch01.keras
199/500 ━━━━━━━━━━━━━━━━━━━━ 7:39 2s/step - accuracy: 0.2539 - loss: 3.0222
Epoch 1: saving model to /content/drive/MyDrive/plant_village_dataset/plant_village_dataset/model_checkpoints/1-27-2025-no-augmentation/mobilenet_epoch01.keras
299/500 ━━━━━━━━━━━━━━━━━━━━ 5:03 2s/step - accuracy: 0.2570 - loss: 2.9843
Epoch 1: saving model to /content/drive/MyDrive/plant_village_dataset/plant_village_dataset/model_checkpoints/1-27-2025-no-augmentation/mobilenet_epoch01.keras
399/500 ━━━━━━━━━━━━━━━━━━━━ 2:30 1s/step - accuracy: 0.2626 - loss: 2.9448
Epoch 1: saving model to /content/drive/MyDrive/plant_village_dataset/plant_village_dataset/model_checkpoints/1-27-2025-no-augmentation/mobilenet_epoch01.keras
499/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accu

KeyboardInterrupt: 